In [0]:

--ALTER TABLE apachedataingestion.nr.widetable_5g_gNB_hr_gold2 ADD COLUMN 	  D_DTM_UTC TIMESTAMP first;


delete from apachedataingestion.nr.widetable_5g_gNB_hr_gold2 
where CalendarDate >= date_format(date_add(now(), int(:days)),"yyyy-MM-dd") and CalendarDate < date_format(date_add(now(), (int(:days))+1),"yyyy-MM-dd") 
;

insert into apachedataingestion.nr.widetable_5g_gNB_hr_gold2
with conf_market (select cast(SUB_MARKET_ID AS INT) SUB_MARKET_ID,SUB_MARKET_NAME,REGION,Time_Offset from hive_metastore.default.vz_conf_market)
select 
case 	when B.Time_Offset=-5 then  convert_timezone('America/New_York','UTC',cast(concat(Y.CalendarDate,' ',lpad(Y.hour24,2,0),':00:00') as TIMESTAMP)) 
	 				when B.Time_Offset=-6 then  convert_timezone('America/Chicago','UTC',cast(concat(Y.CalendarDate,' ',lpad(Y.hour24,2,0),':00:00') as TIMESTAMP)) 
	 				when B.Time_Offset=-8 then  convert_timezone('America/Los_Angeles','UTC',cast(concat(Y.CalendarDate,' ',lpad(Y.hour24,2,0),':00:00') as TIMESTAMP)) 
end D_DTM_UTC,
Y.* 
from (
with conf_gNB (         
						select substring(DU_ID,1,10) GNODEB_ID,max(MARKET_NAME) MARKET_NAME,max(NETWORK_NAME) NETWORK_NAME,max(SUB_MARKET_NAME) SUB_MARKET_NAME,
                        	cast (replace(substring(DU_ID,1,10),'VZ_','') as bigint) kGNODEB_ID		          	
                  			from hive_metastore.`default`.vz_conf_gnode_du_gcell_silver  where not(length(GNODEB_ID) < 10) group by substring(DU_ID,1,10)
						union all						
							select distinct concat('VZ_',gNB_ID) GNODEB_ID,'' MARKET_NAME,'' NETWORK_NAME,'' SUB_MARKET_NAME,cast (gNB_ID as bigint) kGNODEB_ID 							
							from hive_metastore.`default`.jwk_mapping_live a
							full join (SELECT distinct GNODEB_ID FROM hive_metastore.`default`.vz_conf_gnode_du_gcell_silver) b on concat('VZ_',a.gNB_ID)=b.GNODEB_ID
							where b.GNODEB_ID is null 
) 
select 
CalendarDate ,
Hour24 ,
GNODEB_ID,
MARKET_NAME ,
substring(GNODEB_ID,4,3) SUB_MARKET_ID,
SUB_MARKET_NAME,
case when length(GNODEB_ID)=10 then 
	case when substring(GNODEB_ID, 7, 1)=0 then 'FR2' 	 when substring(GNODEB_ID, 7, 1)=9 then 'FR1' 	end
else '' end NR_TYPE,
max(S5NR_DL_PDCP_DataVolume_MB) S5NR_DL_PDCP_DataVolume_MB,
max(S5NR_DL_PDCP_AvgTput_Mbps_Num) S5NR_DL_PDCP_AvgTput_Mbps_Num,
max(S5NR_DL_PDCP_AvgTput_Mbps_Denom) S5NR_DL_PDCP_AvgTput_Mbps_Denom,
max(S5NR_DL_PDCP_Latency_ms_Num) S5NR_DL_PDCP_Latency_ms_Num,
max(S5NR_DL_PDCP_Latency_ms_Denom) S5NR_DL_PDCP_Latency_ms_Denom,
max(S5NR_DL_PDCP_Loss_PCT_Num) S5NR_DL_PDCP_Loss_PCT_Num,
max(S5NR_DL_PDCP_Loss_PCT_Denom) S5NR_DL_PDCP_Loss_PCT_Denom,
max(S5NR_DL_PDCP_Discard_PCT_Num) S5NR_DL_PDCP_Discard_PCT_Num,
max(S5NR_DL_PDCP_Discard_PCT_Denom) S5NR_DL_PDCP_Discard_PCT_Denom,
max(S5NR_DL_PDCP_Drop_PCT_Num) S5NR_DL_PDCP_Drop_PCT_Num,
max(S5NR_DL_PDCP_Drop_PCT_Denom) S5NR_DL_PDCP_Drop_PCT_Denom,
max(S5NR_DL_PDCP_ReTx_PCT_Num) S5NR_DL_PDCP_ReTx_PCT_Num,
max(S5NR_DL_PDCP_ReTx_PCT_Denom) S5NR_DL_PDCP_ReTx_PCT_Denom,
max(S5NR_DL_PDCP_NonSplit_NR_DataVolume_MB) S5NR_DL_PDCP_NonSplit_NR_DataVolume_MB,
max(S5NR_DL_PDCP_NonSplit_NR_AvgTput_Mbps_Num) S5NR_DL_PDCP_NonSplit_NR_AvgTput_Mbps_Num,
max(S5NR_DL_PDCP_NonSplit_NR_AvgTput_Mbps_Denom) S5NR_DL_PDCP_NonSplit_NR_AvgTput_Mbps_Denom,
max(S5NR_DL_PDCP_NonSplit_NR_Latency_ms_Num) S5NR_DL_PDCP_NonSplit_NR_Latency_ms_Num,
max(S5NR_DL_PDCP_NonSplit_NR_Latency_ms_Denom) S5NR_DL_PDCP_NonSplit_NR_Latency_ms_Denom,
max(S5NR_DL_PDCP_Loss_NonSplit_NR_PCT_Num) S5NR_DL_PDCP_Loss_NonSplit_NR_PCT_Num,
max(S5NR_DL_PDCP_Loss_NonSplit_NR_PCT_Denom) S5NR_DL_PDCP_Loss_NonSplit_NR_PCT_Denom,
max(S5NR_DL_PDCP_Split_LTE_DataVolume_MB) S5NR_DL_PDCP_Split_LTE_DataVolume_MB,
max(S5NR_DL_PDCP_Split_LTE_AvgTput_Mbps_Num) S5NR_DL_PDCP_Split_LTE_AvgTput_Mbps_Num,
max(S5NR_DL_PDCP_Split_LTE_AvgTput_Mbps_Denom) S5NR_DL_PDCP_Split_LTE_AvgTput_Mbps_Denom,
max(S5NR_DL_PDCP_Split_LTE_Latency_ms_Num) S5NR_DL_PDCP_Split_LTE_Latency_ms_Num,
max(S5NR_DL_PDCP_Split_LTE_Latency_ms_Denom) S5NR_DL_PDCP_Split_LTE_Latency_ms_Denom,
max(S5NR_DL_PDCP_Loss_Split_LTE_PCT_Num) S5NR_DL_PDCP_Loss_Split_LTE_PCT_Num,
max(S5NR_DL_PDCP_Loss_Split_LTE_PCT_Denom) S5NR_DL_PDCP_Loss_Split_LTE_PCT_Denom,
max(S5NR_DL_PDCP_Split_NR_DataVolume_MB) S5NR_DL_PDCP_Split_NR_DataVolume_MB,
max(S5NR_DL_PDCP_Split_NR_AvgTput_Mbps_Num) S5NR_DL_PDCP_Split_NR_AvgTput_Mbps_Num,
max(S5NR_DL_PDCP_Split_NR_AvgTput_Mbps_Denom) S5NR_DL_PDCP_Split_NR_AvgTput_Mbps_Denom,
max(S5NR_DL_PDCP_Split_NR_Latency_ms_Num) S5NR_DL_PDCP_Split_NR_Latency_ms_Num,
max(S5NR_DL_PDCP_Split_NR_Latency_ms_Denom) S5NR_DL_PDCP_Split_NR_Latency_ms_Denom,
max(S5NR_DL_PDCP_Loss_Split_NR_PCT_Num) S5NR_DL_PDCP_Loss_Split_NR_PCT_Num,
max(S5NR_DL_PDCP_Loss_Split_NR_PCT_Denom) S5NR_DL_PDCP_Loss_Split_NR_PCT_Denom,
max(S5NR_UL_PDCP_DataVolume_MB) S5NR_UL_PDCP_DataVolume_MB,
max(S5NR_UL_PDCP_AvgTput_Mbps_Num) S5NR_UL_PDCP_AvgTput_Mbps_Num,
max(S5NR_UL_PDCP_AvgTput_Mbps_Denom) S5NR_UL_PDCP_AvgTput_Mbps_Denom,
max(S5NR_UL_PDCP_Latency_ms_Num) S5NR_UL_PDCP_Latency_ms_Num,
max(S5NR_UL_PDCP_Latency_ms_Denom) S5NR_UL_PDCP_Latency_ms_Denom,
max(S5NR_UL_PDCP_Loss_PCT_Num) S5NR_UL_PDCP_Loss_PCT_Num,
max(S5NR_UL_PDCP_Loss_PCT_Denom) S5NR_UL_PDCP_Loss_PCT_Denom,
max(S5NR_UL_PDCP_NonSplit_NR_DataVolume_MB) S5NR_UL_PDCP_NonSplit_NR_DataVolume_MB,
max(S5NR_UL_PDCP_NonSplit_NR_AvgTput_Mbps_Num) S5NR_UL_PDCP_NonSplit_NR_AvgTput_Mbps_Num,
max(S5NR_UL_PDCP_NonSplit_NR_AvgTput_Mbps_Denom) S5NR_UL_PDCP_NonSplit_NR_AvgTput_Mbps_Denom,
max(S5NR_UL_PDCP_NonSplit_NR_Latency_ms_Num) S5NR_UL_PDCP_NonSplit_NR_Latency_ms_Num,
max(S5NR_UL_PDCP_NonSplit_NR_Latency_ms_Denom) S5NR_UL_PDCP_NonSplit_NR_Latency_ms_Denom,
max(S5NR_UL_PDCP_Split_LTE_DataVolume_MB) S5NR_UL_PDCP_Split_LTE_DataVolume_MB,
max(S5NR_UL_PDCP_Split_LTE_AvgTput_Mbps_Num) S5NR_UL_PDCP_Split_LTE_AvgTput_Mbps_Num,
max(S5NR_UL_PDCP_Split_LTE_AvgTput_Mbps_Denom) S5NR_UL_PDCP_Split_LTE_AvgTput_Mbps_Denom,
max(S5NR_UL_PDCP_Split_LTE_Latency_ms_Num) S5NR_UL_PDCP_Split_LTE_Latency_ms_Num,
max(S5NR_UL_PDCP_Split_LTE_Latency_ms_Denom) S5NR_UL_PDCP_Split_LTE_Latency_ms_Denom,
max(S5NR_UL_PDCP_Split_NR_DataVolume_MB) S5NR_UL_PDCP_Split_NR_DataVolume_MB,
max(S5NR_UL_PDCP_Split_NR_AvgTput_Mbps_Num) S5NR_UL_PDCP_Split_NR_AvgTput_Mbps_Num,
max(S5NR_UL_PDCP_Split_NR_AvgTput_Mbps_Denom) S5NR_UL_PDCP_Split_NR_AvgTput_Mbps_Denom,
max(S5NR_UL_PDCP_Split_NR_Latency_ms_Num) S5NR_UL_PDCP_Split_NR_Latency_ms_Num,
max(S5NR_UL_PDCP_Split_NR_Latency_ms_Denom) S5NR_UL_PDCP_Split_NR_Latency_ms_Denom,
max(S5NR_F1UPacketLossRate_UL_PCT_Num) S5NR_F1UPacketLossRate_UL_PCT_Num,
max(S5NR_F1UPacketLossRate_UL_PCT_Denom) S5NR_F1UPacketLossRate_UL_PCT_Denom,
max(S5NR_F1UPacketLossRate_DL_PCT_Num) S5NR_F1UPacketLossRate_DL_PCT_Num,
max(S5NR_F1UPacketLossRate_DL_PCT_Denom) S5NR_F1UPacketLossRate_DL_PCT_Denom,
max(DL_PDCP_Volume_Ratio_Num) DL_PDCP_Volume_Ratio_Num,
max(DL_PDCP_Volume_Ratio_Denom) DL_PDCP_Volume_Ratio_Denom,
max(UL_PDCP_Volume_Ratio_Num) UL_PDCP_Volume_Ratio_Num,
max(UL_PDCP_Volume_Ratio_Denom) UL_PDCP_Volume_Ratio_Denom,
-- add
max(S5NU_F1U_PktLoss_UL_SA_ppm_PCT_Num) S5NU_F1U_PktLoss_UL_SA_ppm_PCT_Num,
max(S5NU_F1U_PktLoss_UL_SA_ppm_PCT_Denom) S5NU_F1U_PktLoss_UL_SA_ppm_PCT_Denom,
max(S5NU_PDCP_Datavol_DL_Mb_SA) S5NU_PDCP_Datavol_DL_Mb_SA,
max(S5NU_PDCP_Datavol_UL_Mb_SA) S5NU_PDCP_Datavol_UL_Mb_SA,
max(S5NU_PDCPLoss_UL_SA_ppm_PCT_Num) S5NU_PDCPLoss_UL_SA_ppm_PCT_Num,		
max(S5NU_PDCPLoss_UL_SA_ppm_PCT_Denom) S5NU_PDCPLoss_UL_SA_ppm_PCT_Denom,
max(S5NU_PDCPLoss_DL_SA_ppm_PCT_Num) S5NU_PDCPLoss_DL_SA_ppm_PCT_Num,			
max(S5NU_PDCPLoss_DL_SA_ppm_PCT_Denom) S5NU_PDCPLoss_DL_SA_ppm_PCT_Denom,
max(S5NU_DLPDCPDrop_gNB_ppm_PCT_SA_Num) S5NU_DLPDCPDrop_gNB_ppm_PCT_SA_Num,	
max(S5NU_DLPDCPDrop_gNB_ppm_PCT_SA_Denom) S5NU_DLPDCPDrop_gNB_ppm_PCT_SA_Denom,
max(S5NU_PDCPDelay_DL_SA_us_Num) S5NU_PDCPDelay_DL_SA_us_Num,				
max(S5NU_PDCPDelay_DL_SA_us_Denom) S5NU_PDCPDelay_DL_SA_us_Denom,
MAX(S5NC_InitialContextSetupFailure_PCT_SA_Num) S5NC_InitialContextSetupFailure_PCT_SA_Num,
MAX(S5NC_InitialContextSetupFailure_PCT_SA_Denom)  S5NC_InitialContextSetupFailure_PCT_SA_Denom,

MAX(SEA_DLPDCPLayerDataVol_5QI1_MB) SEA_DLPDCPLayerDataVol_5QI1_MB,
MAX(SEA_DLPDCPLayerDataVol_5QI5_MB) SEA_DLPDCPLayerDataVol_5QI5_MB,
MAX(SEA_VoNR_PdcpIpLatencyDL_5QI1_ms_gNBlevel_Num) SEA_VoNR_PdcpIpLatencyDL_5QI1_ms_gNBlevel_Num,
MAX(SEA_VoNR_PdcpIpLatencyDL_5QI1_ms_gNBlevel_Denom) SEA_VoNR_PdcpIpLatencyDL_5QI1_ms_gNBlevel_Denom,
MAX(SEA_VoNR_PdcpIpLatencyDL_5QI5_ms_gNBlevel_Num) SEA_VoNR_PdcpIpLatencyDL_5QI5_ms_gNBlevel_Num,
MAX(SEA_VoNR_PdcpIpLatencyDL_5QI5_ms_gNBlevel_Denom) SEA_VoNR_PdcpIpLatencyDL_5QI5_ms_gNBlevel_Denom

from ( 
select    		
				COALESCE(M0.CalendarDate,T1.CalendarDate,T2.CalendarDate,T3.CalendarDate,T4.CalendarDate,T5.CalendarDate,T6.CalendarDate) CalendarDate,
				COALESCE(M0.Hour24,T1.Hour24,T2.Hour24,T3.Hour24,T4.Hour24,T5.Hour24,T6.Hour24) Hour24,
				COALESCE(M0.GNODEB_ID,T1.GNODEB_ID,T2.GNODEB_ID,T3.GNODEB_ID,T4.GNODEB_ID,T5.GNODEB_ID,T6.GNODEB_ID) GNODEB_ID,
				COALESCE(M0.MARKET_NAME,T1.MARKET_NAME,T2.MARKET_NAME,T3.MARKET_NAME,T4.MARKET_NAME,T5.MARKET_NAME,T6.MARKET_NAME) MARKET_NAME,        		        		
				COALESCE(M0.SUB_MARKET_NAME,T1.SUB_MARKET_NAME,T2.SUB_MARKET_NAME,T3.SUB_MARKET_NAME,T4.SUB_MARKET_NAME,T5.SUB_MARKET_NAME,T6.SUB_MARKET_NAME) SUB_MARKET_NAME,
				PdcpSduVolumeDL S5NR_DL_PDCP_DataVolume_MB ,
				8 * PdcpSduVolumeDL S5NR_DL_PDCP_AvgTput_Mbps_Num ,
				PdcpSduVolumeDLCnt / 10 S5NR_DL_PDCP_AvgTput_Mbps_Denom ,
				PdcpSduDelayDLTot S5NR_DL_PDCP_Latency_ms_Num ,
				PdcpSduDelayDLCnt S5NR_DL_PDCP_Latency_ms_Denom ,
				PdcpSduLossCntDL S5NR_DL_PDCP_Loss_PCT_Num ,
				PdcpSduCntDL S5NR_DL_PDCP_Loss_PCT_Denom ,
				PdcpSduDiscardCntDL S5NR_DL_PDCP_Discard_PCT_Num ,
				PdcpSduCntDL S5NR_DL_PDCP_Discard_PCT_Denom ,
				PdcpSduDropCntDL S5NR_DL_PDCP_Drop_PCT_Num ,
				PdcpSduCntDL S5NR_DL_PDCP_Drop_PCT_Denom ,
				PdcpSduRetxCntDL S5NR_DL_PDCP_ReTx_PCT_Num ,
				PdcpSduCntDL S5NR_DL_PDCP_ReTx_PCT_Denom ,
				PdcpSduVolumeNonSplitNrDL S5NR_DL_PDCP_NonSplit_NR_DataVolume_MB ,
				8 * PdcpSduVolumeNonSplitNrDL S5NR_DL_PDCP_NonSplit_NR_AvgTput_Mbps_Num ,
				PdcpSduVolumeNonSplitNrDLCnt / 10 S5NR_DL_PDCP_NonSplit_NR_AvgTput_Mbps_Denom ,
				PdcpSduDelayNonSplitNrDLTot S5NR_DL_PDCP_NonSplit_NR_Latency_ms_Num ,
				PdcpSduDelayNonSplitNrDLCnt S5NR_DL_PDCP_NonSplit_NR_Latency_ms_Denom ,
				PdcpSduDelayNonSplitNrDLCnt S5NR_DL_PDCP_Loss_NonSplit_NR_PCT_Num ,
				PdcpSduCntNonSplitNrDL S5NR_DL_PDCP_Loss_NonSplit_NR_PCT_Denom ,
				PdcpSduVolumeSplitLteDL S5NR_DL_PDCP_Split_LTE_DataVolume_MB ,
				8 * PdcpSduVolumeSplitLteDL S5NR_DL_PDCP_Split_LTE_AvgTput_Mbps_Num ,
				PdcpSduVolumeSplitLteDLCnt / 10 S5NR_DL_PDCP_Split_LTE_AvgTput_Mbps_Denom ,
				PdcpSduDelaySplitLteDLTot S5NR_DL_PDCP_Split_LTE_Latency_ms_Num ,
				PdcpSduDelaySplitLteDLCnt S5NR_DL_PDCP_Split_LTE_Latency_ms_Denom ,
				PdcpSduLossCntSplitLteDL S5NR_DL_PDCP_Loss_Split_LTE_PCT_Num ,
				PdcpSduCntSplitLteDL S5NR_DL_PDCP_Loss_Split_LTE_PCT_Denom ,
				PdcpSduVolumeSplitNrDL S5NR_DL_PDCP_Split_NR_DataVolume_MB ,
				8 * PdcpSduVolumeSplitNrDL S5NR_DL_PDCP_Split_NR_AvgTput_Mbps_Num ,
				PdcpSduVolumeSplitNrDLCnt / 10 S5NR_DL_PDCP_Split_NR_AvgTput_Mbps_Denom ,
				PdcpSduDelaySplitNrDLTot S5NR_DL_PDCP_Split_NR_Latency_ms_Num ,
				PdcpSduDelaySplitNrDLCnt S5NR_DL_PDCP_Split_NR_Latency_ms_Denom ,
				PdcpSduLossCntSplitNrDL S5NR_DL_PDCP_Loss_Split_NR_PCT_Num ,
				PdcpSduCntSplitNrDL S5NR_DL_PDCP_Loss_Split_NR_PCT_Denom ,
				PdcpSduVolumeUL S5NR_UL_PDCP_DataVolume_MB ,
				8 * PdcpSduVolumeUL S5NR_UL_PDCP_AvgTput_Mbps_Num ,
				PdcpSduVolumeULCnt / 10 S5NR_UL_PDCP_AvgTput_Mbps_Denom ,
				PdcpSduDelayULTot S5NR_UL_PDCP_Latency_ms_Num ,
				PdcpSduDelayULCnt S5NR_UL_PDCP_Latency_ms_Denom ,
				PdcpSduLossCntUL S5NR_UL_PDCP_Loss_PCT_Num ,
				PdcpSduCntUL S5NR_UL_PDCP_Loss_PCT_Denom ,
				PdcpSduVolumeNonSplitNrUL S5NR_UL_PDCP_NonSplit_NR_DataVolume_MB ,
				8 * PdcpSduVolumeNonSplitNrUL S5NR_UL_PDCP_NonSplit_NR_AvgTput_Mbps_Num ,
				PdcpSduVolumeNonSplitNrULCnt / 10 S5NR_UL_PDCP_NonSplit_NR_AvgTput_Mbps_Denom ,
				PdcpSduDelayNonSplitNrULTot S5NR_UL_PDCP_NonSplit_NR_Latency_ms_Num ,
				PdcpSduDelayNonSplitNrULCnt S5NR_UL_PDCP_NonSplit_NR_Latency_ms_Denom ,
				PdcpSduVolumeSplitLteUL S5NR_UL_PDCP_Split_LTE_DataVolume_MB ,
				8 * PdcpSduVolumeSplitLteUL S5NR_UL_PDCP_Split_LTE_AvgTput_Mbps_Num ,
				PdcpSduVolumeSplitLteULCnt / 10 S5NR_UL_PDCP_Split_LTE_AvgTput_Mbps_Denom ,
				PdcpSduDelaySplitLteULTot S5NR_UL_PDCP_Split_LTE_Latency_ms_Num ,
				PdcpSduDelaySplitLteULCnt S5NR_UL_PDCP_Split_LTE_Latency_ms_Denom ,
				PdcpSduVolumeSplitNrUL S5NR_UL_PDCP_Split_NR_DataVolume_MB ,
				8 * PdcpSduVolumeSplitNrUL S5NR_UL_PDCP_Split_NR_AvgTput_Mbps_Num ,
				PdcpSduVolumeSplitNrULCnt / 10 S5NR_UL_PDCP_Split_NR_AvgTput_Mbps_Denom ,
				PdcpSduDelaySplitNrULTot S5NR_UL_PDCP_Split_NR_Latency_ms_Num ,
				PdcpSduDelaySplitNrULCnt S5NR_UL_PDCP_Split_NR_Latency_ms_Denom ,
				F1UPacketLossCntUL_QCI S5NR_F1UPacketLossRate_UL_PCT_Num ,
				(F1UPacketLossCntUL_QCI + F1UPacketCntUL_QCI) S5NR_F1UPacketLossRate_UL_PCT_Denom ,
				(F1UPacketLossCntDL_QCI - F1UPacketOosCntDL_QCI) S5NR_F1UPacketLossRate_DL_PCT_Num ,
				(F1UPacketLossCntDL_QCI + F1UPacketCntDL_QCI - F1UPacketOosCntDL_QCI) S5NR_F1UPacketLossRate_DL_PCT_Denom,
				PdcpSduVolumeSplitNrDL	DL_PDCP_Volume_Ratio_Num,
				PdcpSduVolumeDL			DL_PDCP_Volume_Ratio_Denom,
				PdcpSduVolumeSplitNrUL	UL_PDCP_Volume_Ratio_Num,
				PdcpSduVolumeUL			UL_PDCP_Volume_Ratio_Denom,
				-- add
				S5NU_F1U_PktLoss_UL_SA_ppm_PCT_Num,
				S5NU_F1U_PktLoss_UL_SA_ppm_PCT_Denom,
				S5NU_PDCP_Datavol_DL_Mb_SA,
				S5NU_PDCP_Datavol_UL_Mb_SA,
				S5NU_PDCPLoss_UL_SA_ppm_PCT_Num,		
				S5NU_PDCPLoss_UL_SA_ppm_PCT_Denom,
				S5NU_PDCPLoss_DL_SA_ppm_PCT_Num	,			
				S5NU_PDCPLoss_DL_SA_ppm_PCT_Denom,
				S5NU_DLPDCPDrop_gNB_ppm_PCT_SA_Num,	
				S5NU_DLPDCPDrop_gNB_ppm_PCT_SA_Denom,
				S5NU_PDCPDelay_DL_SA_us_Num,				
				S5NU_PDCPDelay_DL_SA_us_Denom,
				S5NC_InitialContextSetupFailure_PCT_SA_Num,
				S5NC_InitialContextSetupFailure_PCT_SA_Denom,
				PdcpSduByteDL_5QI1 SEA_DLPDCPLayerDataVol_5QI1_MB,
				PdcpSduByteDL_5QI5 SEA_DLPDCPLayerDataVol_5QI5_MB,
				PdcpSduDelayDLTot_5QI1 SEA_VoNR_PdcpIpLatencyDL_5QI1_ms_gNBlevel_Num,
				PdcpSduDelayDLCnt_5QI1 SEA_VoNR_PdcpIpLatencyDL_5QI1_ms_gNBlevel_Denom,
				PdcpSduDelayDLTot_5QI5 SEA_VoNR_PdcpIpLatencyDL_5QI5_ms_gNBlevel_Num,
				PdcpSduDelayDLCnt_5QI5 SEA_VoNR_PdcpIpLatencyDL_5QI5_ms_gNBlevel_Denom
from (
                  SELECT  Daily CalendarDate,Hourly Hour24,GNODEB_ID,MARKET_NAME, SUB_MARKET_NAME,
	                    sum(F1UPacketLossCntUL_QCI_count_SUM) F1UPacketLossCntUL_QCI,	 
						sum(F1UPacketCntUL_QCI_count_SUM) F1UPacketCntUL_QCI
                        -- select *
                      from  apachedataingestion.nr.f1uulinterfaceperqci_nr_hr_silver a
                      inner join conf_gNB b on cast(left(REGEXP_REPLACE(a.NE_NAME, '[^0-9]', ''),7) as bigint)=b.kGNODEB_ID 
					  WHERE 5=5 and  Daily  =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd") 
                  GROUP BY Daily,Hourly,GNODEB_ID,MARKET_NAME, SUB_MARKET_NAME
                  ) M0         
                  full join  (
					SELECT Daily CalendarDate,Hourly Hour24,GNODEB_ID,MARKET_NAME, SUB_MARKET_NAME,
						sum(PdcpSduVolumeDL_Mbytes_SUM) PdcpSduVolumeDL,
						sum(PdcpSduVolumeDLCnt_count_SUM) PdcpSduVolumeDLCnt,
						sum(PdcpSduVolumeNonSplitNrDL_Mbytes_SUM) PdcpSduVolumeNonSplitNrDL,
						sum(PdcpSduVolumeNonSplitNrDLCnt_count_SUM) PdcpSduVolumeNonSplitNrDLCnt,
						sum(PdcpSduVolumeSplitLteDL_Mbytes_SUM) PdcpSduVolumeSplitLteDL,
						sum(PdcpSduVolumeSplitLteDLCnt_count_SUM) PdcpSduVolumeSplitLteDLCnt,
						sum(PdcpSduVolumeSplitNrDL_Mbytes_SUM) PdcpSduVolumeSplitNrDL,
						sum(PdcpSduVolumeSplitNrDLCnt_count_SUM) PdcpSduVolumeSplitNrDLCnt,
						sum(PdcpSduVolumeUL_Mbytes_SUM) PdcpSduVolumeUL,
						sum(PdcpSduVolumeULCnt_count_SUM) PdcpSduVolumeULCnt,
						sum(PdcpSduVolumeNonSplitNrUL_Mbytes_SUM) PdcpSduVolumeNonSplitNrUL,
						sum(PdcpSduVolumeNonSplitNrULCnt_count_SUM) PdcpSduVolumeNonSplitNrULCnt,
						sum(PdcpSduVolumeSplitLteUL_Mbytes_SUM) PdcpSduVolumeSplitLteUL,
						sum(PdcpSduVolumeSplitLteULCnt_count_SUM) PdcpSduVolumeSplitLteULCnt,
						sum(PdcpSduVolumeSplitNrUL_Mbytes_SUM) PdcpSduVolumeSplitNrUL,
						sum(PdcpSduVolumeSplitNrULCnt_count_SUM) PdcpSduVolumeSplitNrULCnt
						-- select *
					from apachedataingestion.nr.pdcpvolumepergnbid_hr_silver  a
					inner join conf_gNB b on cast(replace(split(a.LOCATION,'/')[1],'gNB_ID','') as bigint)=b.kGNODEB_ID
					WHERE 5=5 and  Daily  =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd") 
					GROUP BY  Daily,Hourly,GNODEB_ID,MARKET_NAME, SUB_MARKET_NAME
			) T1 on M0.CalendarDate=T1.CalendarDate and M0.Hour24=T1.Hour24 and M0.GNODEB_ID=T1.GNODEB_ID 
			full join  (
					SELECT Daily CalendarDate,Hourly Hour24,GNODEB_ID,MARKET_NAME, SUB_MARKET_NAME,
					sum(PdcpSduDelayDLTot_ms_SUM) PdcpSduDelayDLTot,
					sum(PdcpSduLossCntDL_count_SUM) PdcpSduLossCntDL,
					sum(PdcpSduDiscardCntDL_count_SUM) PdcpSduDiscardCntDL,
					sum(PdcpSduDropCntDL_count_SUM) PdcpSduDropCntDL,
					sum(PdcpSduRetxCntDL_count_SUM) PdcpSduRetxCntDL,
					sum(PdcpSduDelayNonSplitNrDLTot_ms_SUM) PdcpSduDelayNonSplitNrDLTot,
					sum(PdcpSduDelayNonSplitNrDLCnt_count_SUM) PdcpSduDelayNonSplitNrDLCnt,
					sum(PdcpSduDelaySplitLteDLTot_ms_SUM) PdcpSduDelaySplitLteDLTot,
					sum(PdcpSduLossCntSplitLteDL_count_SUM) PdcpSduLossCntSplitLteDL,
					sum(PdcpSduDelaySplitNrDLTot_ms_SUM) PdcpSduDelaySplitNrDLTot,
					sum(PdcpSduLossCntSplitNrDL_count_SUM) PdcpSduLossCntSplitNrDL,
					sum(PdcpSduDelayULTot_ms_SUM) PdcpSduDelayULTot,
					sum(PdcpSduLossCntUL_count_SUM) PdcpSduLossCntUL,
					sum(PdcpSduDelayNonSplitNrULTot_ms_SUM) PdcpSduDelayNonSplitNrULTot,
					sum(PdcpSduDelaySplitLteULTot_ms_SUM) PdcpSduDelaySplitLteULTot,
					sum(PdcpSduDelaySplitNrULTot_ms_SUM) PdcpSduDelaySplitNrULTot,
					sum(PdcpSduDelayDLCnt_count_SUM) PdcpSduDelayDLCnt,
					sum(PdcpSduCntDL_count_SUM) PdcpSduCntDL,
					sum(PdcpSduCntNonSplitNrDL_count_SUM) PdcpSduCntNonSplitNrDL,
					sum(PdcpSduDelaySplitLteDLCnt_count_SUM) PdcpSduDelaySplitLteDLCnt,
					sum(PdcpSduCntSplitLteDL_count_SUM) PdcpSduCntSplitLteDL,
					sum(PdcpSduDelaySplitNrDLCnt_count_SUM) PdcpSduDelaySplitNrDLCnt,
					sum(PdcpSduCntSplitNrDL_count_SUM) PdcpSduCntSplitNrDL,
					sum(PdcpSduDelayULCnt_count_SUM) PdcpSduDelayULCnt,
					sum(PdcpSduCntUL_count_SUM) PdcpSduCntUL,
					sum(PdcpSduDelayNonSplitNrULCnt_count_SUM) PdcpSduDelayNonSplitNrULCnt,
					sum(PdcpSduDelaySplitLteULCnt_count_SUM) PdcpSduDelaySplitLteULCnt,
					sum(PdcpSduDelaySplitNrULCnt_count_SUM) PdcpSduDelaySplitNrULCnt
					-- select *
					from apachedataingestion.nr.pdcppacketpergnbid_hr_silver  a
					inner join conf_gNB b on cast(replace(split(a.LOCATION,'/')[1],'gNB_ID','') as bigint)=b.kGNODEB_ID
					WHERE 5=5 and  Daily  =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd") 
					GROUP BY  Daily,Hourly,GNODEB_ID,MARKET_NAME, SUB_MARKET_NAME
			) T2 on M0.CalendarDate=T2.CalendarDate and M0.Hour24=T2.Hour24 and M0.GNODEB_ID=T2.GNODEB_ID
			full join  (
					SELECT  Daily CalendarDate,Hourly Hour24,GNODEB_ID,MARKET_NAME, SUB_MARKET_NAME,
	                    sum(F1UPacketLossCntDL_QCI_count_SUM) F1UPacketLossCntDL_QCI,
						sum(F1UPacketCntDL_QCI_count_SUM) F1UPacketCntDL_QCI,
						sum(F1UPacketOosCntDL_QCI_count_SUM) F1UPacketOosCntDL_QCI
                        -- select *
                      from  apachedataingestion.nr.f1udlinterfaceperqci_nr_hr_silver a
                      inner join conf_gNB b on cast(left(REGEXP_REPLACE(a.NE_NAME, '[^0-9]', ''),7) as bigint)=b.kGNODEB_ID 
					  WHERE 5=5 and  Daily  =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd") 
                  GROUP BY Daily,Hourly,GNODEB_ID,MARKET_NAME, SUB_MARKET_NAME
		   ) T3 on M0.CalendarDate=T3.CalendarDate and M0.Hour24=T3.Hour24 and M0.GNODEB_ID=T3.GNODEB_ID
		   full join  (
					SELECT  Daily CalendarDate,Hourly Hour24,GNODEB_ID,MARKET_NAME, SUB_MARKET_NAME,
	                     sum(F1UPacketLossCntUL * power(10, 6))  S5NU_F1U_PktLoss_UL_SA_ppm_PCT_Num,
						 sum((F1UPacketCntUL + F1UPacketLossCntUL)) S5NU_F1U_PktLoss_UL_SA_ppm_PCT_Denom
                        -- select *
                      from  apachedataingestion.nr.f1uulinterfaceperupc_nr_hr_silver a
                      inner join conf_gNB b on cast(left(REGEXP_REPLACE(a.NE_NAME, '[^0-9]', ''),7) as bigint)=b.kGNODEB_ID 
					  WHERE 5=5 and  Daily  =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd") 
                  GROUP BY Daily,Hourly,GNODEB_ID,MARKET_NAME, SUB_MARKET_NAME
		   ) T4 on M0.CalendarDate=T4.CalendarDate and M0.Hour24=T4.Hour24 and M0.GNODEB_ID=T4.GNODEB_ID
		    full join  (
					SELECT  Daily CalendarDate,Hourly Hour24,GNODEB_ID,MARKET_NAME, SUB_MARKET_NAME,
							sum(PdcpSduByteDL * power(1.024, 2))	S5NU_PDCP_Datavol_DL_Mb_SA,
							sum(PdcpSduByteUL * power(1.024, 2))	S5NU_PDCP_Datavol_UL_Mb_SA,
							sum(PdcpSduLossCntUL * power(10, 6)) S5NU_PDCPLoss_UL_SA_ppm_PCT_Num	,		
							sum(PdcpSduLossCntUL + PdcpSduCntUL)  S5NU_PDCPLoss_UL_SA_ppm_PCT_Denom,
							sum(PdcpSduLossCntDL * power(10, 6)) S5NU_PDCPLoss_DL_SA_ppm_PCT_Num	,			
							sum(PdcpSduCntDL) S5NU_PDCPLoss_DL_SA_ppm_PCT_Denom,
							sum(PdcpSduDiscardCntDL + PdcpSduDropCntDL) * power(10, 6)	S5NU_DLPDCPDrop_gNB_ppm_PCT_SA_Num,	
							sum(PdcpSduCntDL) S5NU_DLPDCPDrop_gNB_ppm_PCT_SA_Denom,
							sum(1000 * PdcpSduDelayDLTot)	S5NU_PDCPDelay_DL_SA_us_Num,				
							sum(PdcpSduDelayDLCnt) S5NU_PDCPDelay_DL_SA_us_Denom,
							sum(case when split(LOCATION,'/')[2] in ('5QI1') then nvl(PdcpSduByteDL ,0)  * 1.024 * 1.024 end )  PdcpSduByteDL_5QI1,
							sum(case when split(LOCATION,'/')[2] in ('5QI1') then nvl(PdcpSduDelayDLTot,0) end )  PdcpSduDelayDLTot_5QI1,
							sum(case when split(LOCATION,'/')[2] in ('5QI1') then nvl(PdcpSduDelayDLCnt,0) end )  PdcpSduDelayDLCnt_5QI1,
							sum(case when split(LOCATION,'/')[2] in ('5QI5') then nvl(PdcpSduByteDL ,0)  * 1.024 * 1.024 end )  PdcpSduByteDL_5QI5,
							sum(case when split(LOCATION,'/')[2] in ('5QI5') then nvl(PdcpSduDelayDLTot,0) end )  PdcpSduDelayDLTot_5QI5,
							sum(case when split(LOCATION,'/')[2] in ('5QI5') then nvl(PdcpSduDelayDLCnt,0) end )  PdcpSduDelayDLCnt_5QI5
                        -- select *
                      from  apachedataingestion.nr.pdcppacketpergnbidper5qi_nr_hr_silver a
					  inner join conf_gNB b on cast(replace(split(a.LOCATION,'/')[1],'gNB_ID','') as bigint)=b.kGNODEB_ID
                      --inner join conf_gNB b on cast(left(REGEXP_REPLACE(a.NE_NAME, '[^0-9]', ''),7) as bigint)=b.kGNODEB_ID 
					  WHERE 5=5 and  Daily  =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd") 
                  GROUP BY Daily,Hourly,GNODEB_ID,MARKET_NAME, SUB_MARKET_NAME
		   ) T5 on M0.CalendarDate=T5.CalendarDate and M0.Hour24=T5.Hour24 and M0.GNODEB_ID=T5.GNODEB_ID
		   full join  (
					SELECT  Daily CalendarDate,Hourly Hour24,GNODEB_ID,MARKET_NAME, SUB_MARKET_NAME,
							sum(Tx_NGInitialContextSetupFailure)	S5NC_InitialContextSetupFailure_PCT_SA_Num,
							sum(Rx_NGInitialContextSetupRequest)	S5NC_InitialContextSetupFailure_PCT_SA_Denom
							-- select *
                      from  apachedataingestion.nr.UEassociatedNGAPMessageingNB_nr_hr_silver a
					  inner join conf_gNB b on cast(replace(split(a.LOCATION,'/')[0],'gNB_ID','') as bigint)=b.kGNODEB_ID
                      --inner join conf_gNB b on cast(left(REGEXP_REPLACE(a.NE_NAME, '[^0-9]', ''),7) as bigint)=b.kGNODEB_ID 
					  WHERE 5=5 and  Daily  =  date_format(date_add(now(), int(:days)),"yyyy-MM-dd") 
                  GROUP BY Daily,Hourly,GNODEB_ID,MARKET_NAME, SUB_MARKET_NAME
		   ) T6 on M0.CalendarDate=T6.CalendarDate and M0.Hour24=T6.Hour24 and M0.GNODEB_ID=T6.GNODEB_ID
) X          
Group by CalendarDate ,Hour24 ,GNODEB_ID,MARKET_NAME  ,SUB_MARKET_ID,SUB_MARKET_NAME 
) Y
inner join conf_market B on cast(Y.SUB_MARKET_ID AS INT)=B.SUB_MARKET_ID	
;


